In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cycler
from scipy import stats
import statsmodels.api as sm

In [3]:
df1 = pd.read_csv('25_Portfolios_5x5.csv')
df2 = pd.read_csv('F-F_Research_Data_Factors.csv')
df = pd.merge(df1, df2, on = 'month')
df.head()

,month,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM,Mkt-RF,SMB,HML,RF
0,192607,3.7782,-0.4119,-1.9434,0.3530,2.0534,2.1904,2.4192,0.4926,-1.5770,...,2.4678,3.4539,6.0902,2.0266,3.1111,0.5623,2.96,-2.30,-2.87,0.22
1,192608,-2.2074,-8.7275,2.4404,0.6086,8.3968,2.1709,-1.1849,4.0084,0.4643,...,5.3422,1.0124,4.1903,2.0131,5.4849,7.7576,2.64,-1.40,4.19,0.25
2,192609,-6.2113,-0.2989,-6.1982,-1.6368,0.8649,-1.8550,-1.2618,1.0829,-3.0405,...,0.8730,-1.2906,3.6538,0.0950,-0.7487,-2.4284,0.36,-1.32,0.01,0.23
3,192610,-8.6241,-3.7532,-5.6719,5.7170,-2.5476,-1.7995,-3.2663,-5.0745,-8.0450,...,-5.3525,-2.7413,-3.0071,-2.2437,-4.6719,-5.8129,-3.24,0.04,0.51,0.32
4,192611,3.4744,6.6476,2.2634,-4.7020,0.5362,2.9051,-2.3690,3.0078,4.6649,...,1.8213,4.2946,2.5326,1.5204,3.6619,2.5636,2.53,-0.20,-0.35,0.31


In [4]:
ExcessReturns = df.iloc[:, 1:26].sub(df['RF'], axis=0)
ExcessReturns.head()

,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
0,3.5582,-0.6319,-2.1634,0.1330,1.8334,1.9704,2.1992,0.2726,-1.7970,-0.5544,...,1.3693,1.3078,0.9669,0.0527,2.2478,3.2339,5.8702,1.8066,2.8911,0.3423
1,-2.4574,-8.9775,2.1904,0.3586,8.1468,1.9209,-1.4349,3.7584,0.2143,5.9175,...,1.0836,3.6230,1.7559,1.9206,5.0922,0.7624,3.9403,1.7631,5.2349,7.5076
2,-6.4413,-0.5289,-6.4282,-1.8668,0.6349,-2.0850,-1.4918,0.8529,-3.2705,-1.1701,...,0.8623,-0.7550,-1.9614,1.2346,0.6430,-1.5206,3.4238,-0.1350,-0.9787,-2.6584
3,-8.9441,-4.0732,-5.9919,5.3970,-2.8676,-2.1195,-3.5863,-5.3945,-8.3650,-1.6413,...,-3.6561,-2.9759,-2.3516,-3.4251,-5.6725,-3.0613,-3.3271,-2.5637,-4.9919,-6.1329
4,3.1644,6.3376,1.9534,-5.0120,0.2262,2.5951,-2.6790,2.6978,4.3549,2.4192,...,3.1348,2.0787,3.4303,4.6220,1.5113,3.9846,2.2226,1.2104,3.3519,2.2536


In [43]:
alpha = []
beta = []
residual = []
for j in range(25):
    Y = np.array(ExcessReturns.iloc[:, j])[:, np.newaxis]
    # Independent variables (nparray)
    X = np.array(df['Mkt-RF'])[:, np.newaxis]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X)
    result = model.fit(cov_type = 'HC0')
    alpha.append(result.params[0])
    beta.append(result.params[1])
    residual.append(result.resid)
alpha = np.array(alpha)[:, np.newaxis]
beta = np.array(beta)[:, np.newaxis]
residual = np.array(residual)

In [26]:
# GRS Test
T = ExcessReturns.shape[0]
sigma = (residual @ residual.transpose()) / T
GRS_stat = T / (1 + df['Mkt-RF'].mean() / df['Mkt-RF'].var() * df['Mkt-RF'].mean()) * (alpha.transpose() @ np.linalg.inv(sigma) @ alpha)

In [28]:
GRS_stat[0, 0]

83.49405349694516

In [30]:
1 - stats.chi2.cdf(GRS_stat[0, 0], 25)

3.201927423202022e-08

In [33]:
D = - np.kron(np.array([[1, df['Mkt-RF'].mean()],
                        [df['Mkt-RF'].mean(), (df['Mkt-RF'] * df['Mkt-RF']).mean()]]), np.eye(25))

In [40]:
cov_alpha = (1 + (df['Mkt-RF'].mean()) * (df['Mkt-RF'].mean()) / df['Mkt-RF'].var()) / T * sigma
alpha.transpose() @ np.linalg.inv(cov_alpha) @ alpha

array([[83.4940535]])

In [58]:
cov_beta = sigma / (df['Mkt-RF'].var() * T)

In [59]:
gamma = np.zeros((25, 1))
gamma[0, 0] = -1
gamma[20, 0] = 1
print(gamma.transpose() @ beta)
print(gamma.transpose() @ beta / np.sqrt(gamma.transpose() @ cov_beta @ gamma))

[[-0.66152672]]
[[-13.34666261]]


In [54]:
gamma = np.zeros((25, 1))
gamma[1, 0] = -1
gamma[21, 0] = 1
print(gamma.transpose() @ beta)
print(gamma.transpose() @ beta / np.sqrt(gamma.transpose() @ cov_beta @ gamma))

[[-0.45537079]]
[[-11.87705164]]


In [55]:
gamma = np.zeros((25, 1))
gamma[2, 0] = -1
gamma[22, 0] = 1
print(gamma.transpose() @ beta)
print(gamma.transpose() @ beta / np.sqrt(gamma.transpose() @ cov_beta @ gamma))

[[-0.39694692]]
[[-12.5649493]]


In [56]:
gamma = np.zeros((25, 1))
gamma[3, 0] = -1
gamma[23, 0] = 1
print(gamma.transpose() @ beta)
print(gamma.transpose() @ beta / np.sqrt(gamma.transpose() @ cov_beta @ gamma))

[[-0.16249091]]
[[-5.72729286]]


In [57]:
gamma = np.zeros((25, 1))
gamma[4, 0] = -1
gamma[24, 0] = 1
print(gamma.transpose() @ beta)
print(gamma.transpose() @ beta / np.sqrt(gamma.transpose() @ cov_beta @ gamma))

[[-0.06636202]]
[[-1.94379522]]


## Cross-sectional regressions

### OLS

In [68]:
Y = np.array(ExcessReturns.mean())[:, np.newaxis]
# Independent variables (nparray)
X = beta
model = sm.OLS(Y, X)
result = model.fit(cov_type = 'HC0')
print(result.params[0], result.tvalues[0])

0.7155755613995083 20.560999230579363


### Fama-MacBeth

In [86]:
lambda_month = []
lambda_std_month = []
lambda_residual = []
for i in range(ExcessReturns.shape[0]):
    Y = np.array(ExcessReturns.iloc[i, :])[:, np.newaxis]
    # Independent variables (nparray)
    X = beta
    model = sm.OLS(Y, X)
    result = model.fit(cov_type = 'HC0')
    lambda_month.append(result.params[0])
    lambda_std_month.append(result.bse[0])
    lambda_residual.append(result.resid)

In [75]:
lambda_FMB = np.array(lambda_month).mean()
lambda_FMB

0.7155755613995085

In [116]:
var_lambda_FMB = np.array(lambda_month).var() / T

In [117]:
lambda_FMB / np.sqrt(var_lambda_FMB)

4.2089023266371255

In [95]:
alpha = pd.DataFrame(np.array(lambda_residual)).mean()
alpha_cov = pd.DataFrame(np.array(lambda_residual)).cov() / T
np.array(alpha)[np.newaxis, :] @ np.linalg.inv(np.array(alpha_cov)) @ np.array(alpha)[: np.newaxis]

array([83.70558552])

In [118]:
lambda0_month = []
lambda1_month = []
for i in range(ExcessReturns.shape[0]):
    Y = np.array(ExcessReturns.iloc[i, :])[:, np.newaxis]
    # Independent variables (nparray)
    X = beta
    X = sm.add_constant(X)
    model = sm.OLS(Y, X)
    result = model.fit(cov_type = 'HC0')
    lambda0_month.append(result.params[0])
    lambda1_month.append(result.params[1])
    
print(np.array(lambda0_month).mean(),
      np.array(lambda0_month).mean() / np.sqrt(np.array(lambda0_month).var() / T))

print(np.array(lambda1_month).mean(),
      np.array(lambda1_month).mean() / np.sqrt(np.array(lambda1_month).var() / T))

0.39179052627027966 1.2171800297482893
0.39964710508451085 1.142839103989548


## GMM

In [131]:
A = np.zeros((51, 75))
np.fill_diagonal(A, 1)
A[50, 50] = 0
A[50, 50:] = beta[:, 0]

In [150]:
D_ = np.array([[1, df['Mkt-RF'].mean()],
              [df['Mkt-RF'].mean(), (df['Mkt-RF'] * df['Mkt-RF']).mean()],
              [0, lambda_FMB]])
D_ = np.kron(D_, np.eye(25))
D = np.zeros((75, 51))
D[:75, :50] = D_
D[50:, 50] = beta[:, 0]
D = -D

In [206]:
moment1 = np.array(ExcessReturns).transpose() - np.array(alpha)[:, np.newaxis] - beta * np.array(df['Mkt-RF'])[np.newaxis, :]

In [207]:
moment2 = (np.array(ExcessReturns).transpose() - np.array(alpha)[:, np.newaxis] - beta * np.array(df['Mkt-RF'])[np.newaxis, :]) * np.array(df['Mkt-RF'])[np.newaxis, :]

In [208]:
moment3 = np.array(ExcessReturns).transpose() - beta * lambda_FMB

In [209]:
moment = np.concatenate((moment1,moment2, moment3),axis=0)

In [210]:
S = moment @ moment.transpose() / T

In [211]:
cov_b = (1/T) * np.linalg.inv(A @ D) @ A @ S @ A.transpose() @ (np.linalg.inv(A @ D)).transpose()

In [212]:
cov_b[-1, -1]

0.026104493085800193

In [213]:
lambda_FMB / np.sqrt(cov_b[-1, -1])

4.428920328409146

In [214]:
pd.DataFrame(moment3.transpose()).mean()

0    -0.566544
1    -0.302816
2    -0.001819
3     0.227500
4     0.328353
5    -0.258409
6     0.054253
7     0.108554
8     0.155219
9     0.218613
10   -0.157844
11    0.102941
12    0.100854
13    0.175074
14    0.086880
15   -0.031045
16   -0.005788
17    0.052046
18    0.101102
19   -0.003482
20   -0.013043
21   -0.045566
22    0.009112
23   -0.150779
24   -0.020340
dtype: float64

In [215]:
np.sqrt(T) * pd.DataFrame(moment3.transpose()).mean() / pd.DataFrame(moment3.transpose()).std()

0    -1.574371
1    -1.038979
2    -0.006845
3     0.920386
4     1.195197
5    -1.090246
6     0.243513
7     0.504929
8     0.704415
9     0.845481
10   -0.717804
11    0.534899
12    0.522927
13    0.851641
14    0.344055
15   -0.167107
16   -0.032018
17    0.273507
18    0.497015
19   -0.013508
20   -0.082136
21   -0.290001
22    0.054522
23   -0.768457
24   -0.079961
dtype: float64

In [220]:
cov_gb = (1/T) * (np.eye(75) - D @ np.linalg.inv(A @ D) @ A) @ S @ (np.eye(75) - D @ np.linalg.inv(A @ D) @ A).transpose()

In [221]:
moment.mean(axis = 1)[np.newaxis, :] @ np.linalg.pinv(cov_gb) @ moment.mean(axis = 1)[:, np.newaxis]

array([[78.9646345]])

In [234]:
lags = 12
S_nw = moment @ moment.transpose() / T
for j in range(1, lags + 1):
    S_nw += (2 * moment[:, :-j] @ moment[:, j:].transpose() / (T - j)) * (lags + 1 - j) / (lags + 1)

In [235]:
cov_b = (1/T) * np.linalg.inv(A @ D) @ A @ S_nw @ A.transpose() @ (np.linalg.inv(A @ D)).transpose()
lambda_FMB / np.sqrt(cov_b[-1, -1])

4.178448872024355